# References

In [1]:
# http://theautomatic.net/yahoo_fin-documentation/
# http://theautomatic.net/2020/05/05/how-to-download-fundamentals-data-with-python/
# https://algotrading101.com/learn/yahoo-finance-api-guide/>
# https://blog.quantinsti.com/quantitative-value-investing-strategy-python/
# https://www.activestate.com/blog/top-10-python-packages-for-finance-and-financial-modeling/
# https://medium.com/automation-generation/teaching-your-computer-to-invest-with-python-commission-free-automated-investing-5ade10961e08

# Imports

In [2]:
import pandas as pd
import yahoo_fin.stock_info as si
import time
import timeit

start = timeit.default_timer()

# Variables

In [3]:
ratio_valuation_function=['Price/Book (mrq)','Trailing P/E','Forward P/E 1',\
                          'PEG Ratio (5 yr expected) 1','Price/Sales (ttm)']

ratio_stat=['Total Debt/Equity (mrq)', 'Diluted EPS (ttm)', 'Trailing Annual Dividend Yield 3',\
            'Forward Annual Dividend Yield 4', '% Held by Insiders 1','% Held by Institutions 1',\
            'Return on Equity (ttm)','Return on Assets (ttm)','Quarterly Earnings Growth (yoy)',\
            'current_price']

# ratio_stat=['Total Debt/Equity (mrq)', 'Diluted EPS (ttm)', 'Trailing Annual Dividend Yield 3',\
#             'Forward Annual Dividend Yield 4', '% Held by Insiders 1','% Held by Institutions 1',\
#             'Return on Equity (ttm)','Return on Assets (ttm)','Quarterly Earnings Growth (yoy)', \
#             'current_price','Beta (5Y Monthly)']

# Get Updated S&P 500

In [4]:
# complete sp 500 list
# file generated: get_sp_500_update.py
# ticker_df = pd.read_csv('../ProjectDatasets/sp_500_symbols.csv', sep=',')
# tickers = ticker_df['Symbol'].tolist()

# get latest sp500
tickers = si.tickers_sp500()

# tickersDF = si.tickers_sp500(True)
# tickers = tickers[:4].copy()

print(len(tickers))

505


# Company Info

In [5]:
tickersDF = si.tickers_sp500(True)
tickersDF.index = tickersDF['Symbol']

tickersDF.index.rename('ticker', inplace=True)
tickersDF.drop(columns=['Symbol'], inplace=True)

# Sort Data

In [6]:
table=pd.DataFrame()

ticker_index = []
retry_ticker = []
count = 0

for p in tickers:
#     print(p)
    try:
        data=si.get_stats(p)
        data.index=data["Attribute"]
        data=data.drop(labels="Attribute",axis=1)
        raw_table=data.T
        raw_table['current_price'] = round(si.get_live_price(p),2)
        table=table.append(raw_table)   #Table having Data about the company
        ticker_index.append(p)
    except:
        count = count+1
        print('Bad Ticker {}: {}'.format(count, p))
        retry_ticker.append(p)

if len(retry_ticker) > 0:     
    time.sleep(60*20)
    count = 0

    for p in retry_ticker:
    #     print(p)
        try:
            data=si.get_stats(p)
            data.index=data["Attribute"]
            data=data.drop(labels="Attribute",axis=1)
            raw_table=data.T
            raw_table['current_price'] = round(si.get_live_price(p),2)
            table=table.append(raw_table)   #Table having Data about the company
            ticker_index.append(p)
            time.sleep(10)
        except:
            count = count+1
            print('Bad Ticker 2nd attempt {}: {}'.format(count, p))
            time.sleep(60*5)
    
table.index=ticker_index
table1 = table[ratio_stat]

In [7]:
time.sleep(60*20)

In [8]:
table=pd.DataFrame()

tickers = ticker_index.copy()
new_index = []
retry_ticker = []
count = 0

for p in tickers:
#     print(p)
    try:
        extra_ratio=si.get_stats_valuation(p)
        extra_ratio = extra_ratio.iloc[:,0:2]
        extra_ratio.index=extra_ratio['Unnamed: 0']
        extra_ratio=extra_ratio.drop(labels='Unnamed: 0',axis=1)
        new_table=extra_ratio.T
        table=table.append(new_table)  #Table having Data about the company
        new_index.append(p)
        time.sleep(10)
    except:
        count = count+1
        print('Bad Ticker {}: {}'.format(count, p))
        retry_ticker.append(p)
        time.sleep(60*5)
        
if len(retry_ticker) > 0:     
    time.sleep(60*20)
    count = 0

    for p in retry_ticker:
    #     print(p)
        try:
            data=si.get_stats(p)
            data.index=data["Attribute"]
            data=data.drop(labels="Attribute",axis=1)
            raw_table=data.T
            raw_table['current_price'] = round(si.get_live_price(p),2)
            table=table.append(raw_table)   #Table having Data about the company
            new_index.append(p)
            time.sleep(10)
        except:
            count = count+1
            print('Bad Ticker 2nd attempt {}: {}'.format(count, p))
            time.sleep(60*5)
        
table.index=new_index
table2 = table[ratio_valuation_function]

In [9]:
final=pd.concat([table2,table1],axis=1)
# og_final = final.copy()

In [10]:
final.to_csv('../ProjectDatasets/final_recommendations_int.csv', index=True, index_label='ticker')
print(final.shape)

(505, 15)


# Evaluations

In [11]:
final['Trailing P/E'] = pd.to_numeric(final['Trailing P/E'], errors='coerce')
final['Price/Book (mrq)'] = pd.to_numeric(final['Price/Book (mrq)'], errors='coerce')

In [12]:
# low_valuations
final = final[(final['Trailing P/E'].astype(float)<40) & (final['Price/Book (mrq)'].astype(float) < 15)].copy()

# earning_power
final = final[final['Diluted EPS (ttm)'].astype(float) > 4].copy()

# equity_to_debt
final = final[(final['Total Debt/Equity (mrq)'].astype(float)< 75 )].copy() # Filter for Debt to Equity
final = final[(final['Return on Equity (ttm)'] > str(20) )].copy() # Filter for ROE

# insider_owned
final = final[final['% Held by Insiders 1']>str(.07)].copy()

In [13]:
FINAL = pd.concat([tickersDF,final], axis=1, join='inner')
FINAL.sort_values(by=['current_price'], inplace=True)

In [14]:
FINAL.head()

,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded,Price/Book (mrq),Trailing P/E,...,Total Debt/Equity (mrq),Diluted EPS (ttm),Trailing Annual Dividend Yield 3,Forward Annual Dividend Yield 4,% Held by Insiders 1,% Held by Institutions 1,Return on Equity (ttm),Return on Assets (ttm),Quarterly Earnings Growth (yoy),current_price
PHM,PulteGroup,reports,Consumer Discretionary,Homebuilding,"Atlanta, Georgia",1984-04-30,822416,1956,2.35,10.74,...,50.85,5.18,0.80%,0.94%,3.50%,90.66%,23.39%,9.68%,30.50%,59.12
CNC,Centene Corporation,reports,Health Care,Managed Health Care,"St. Louis, Missouri",2016-03-30,1071739,1984,1.34,14.32,...,69.07,4.19,NaN,NaN,1.15%,94.56%,9.74%,4.33%,"1,419.60%",61.74
HOLX,Hologic,reports,Health Care,Health Care Equipment,"Marlborough, Massachusetts",2016-03-30,859737,1985,5.22,12.54,...,73.74,7.28,NaN,NaN,0.60%,97.52%,64.65%,21.16%,543.70%,65.55
DHI,D. R. Horton,reports,Consumer Discretionary,Homebuilding,"Arlington, Texas",2005-06-22,882184,1978,2.82,11.77,...,34.32,8.63,0.74%,0.79%,10.01%,83.60%,26.60%,13.30%,92.60%,98.29
PGR,Progressive Corp.,reports,Financials,Property & Casualty Insurance,"Mayfield Village, Ohio",1997-08-04,80661,1937,3.34,9.27,...,30.26,11.00,4.81%,0.39%,0.30%,84.04%,40.02%,8.62%,111.70%,100.74


In [15]:
FINAL.to_csv('../ProjectDatasets/final_recommendations.csv', index=True, index_label='ticker')

In [16]:
FINAL.shape

(42, 23)

In [17]:
stop = timeit.default_timer()

print('Time (hrs): ', ((stop - start)/60)/60)  

Time (hrs):  2.2048174439722223
